In [ ]:
# Import needed libraries

import torch as th
import numpy as np
from random import randint
from random import choice
from random import uniform
from random import shuffle
from scipy.stats import multivariate_normal
from scipy.special import comb
from scipy.special import binom
from math import floor
from math import exp
import csv

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd

import matplotlib.pyplot as plt

from numba import jit

In [ ]:
# Import the graphlet equivalence class data from MATLAB

matlab_data_3 = scipy.io.loadmat('classes3.mat')
matlab_data_4 = scipy.io.loadmat('classes4.mat')
matlab_data_5 = scipy.io.loadmat('classes5.mat')
matlab_data_6 = scipy.io.loadmat('classes6.mat')

In [ ]:
# These identify which equivalence class each graph belongs to (enumerated by binary counting from 0 to n*(n-1)/2 -1)
classes3 = matlab_data_3['classes_master']
classes4 = matlab_data_4['classes_master']
classes5 = matlab_data_5['classes_master']
classes6 = matlab_data_6['classes_master']

# These are a list of an example graph belonging to each equivalence class
graphs3 = matlab_data_3['all_graphs']
graphs4 = matlab_data_4['all_graphs']
graphs5 = matlab_data_5['all_graphs']
graphs6 = matlab_data_6['all_graphs']

# These are the reverse of classesX. They list the graphs that belong to each equivalence class
reverse3 = matlab_data_3['reverse_master']
reverse4 = matlab_data_4['reverse_master']
reverse5 = matlab_data_5['reverse_master']
reverse6 = matlab_data_6['reverse_master']

# Total number of graphlet equivalence classes of each size. 
num_classes_3 = graphs3.shape[2]
num_classes_4 = graphs4.shape[2]
num_classes_5 = graphs5.shape[2]
num_classes_6 = graphs6.shape[2]
#num_classes_high = graphs_high.shape[2]
num_classes_total = num_classes_3 + num_classes_4 + num_classes_5 + num_classes_6
#num_classes_total = num_classes_3 + num_classes_4 + num_classes_5 + num_classes_6 + num_classes_high

# = 4 + 11 + 34 + 156 + X = 205 + X


In [1]:
# This section creates the neural network that does all the work in this problem

class graph_learner(nn.Module):
    def __init__(self, Hbars, max_nodes, num_features, kern, kern_deriv, multi=0, multi_probs=[], max_nodes_multi=0, dropout=0):
        super(graph_learner, self).__init__()
        
        # input parameter Hbars is empirical proportions of graphlets as a vector 205 x 1. First 4 entries are the 3-graphlets, next 11 are the 4-graphlets...
        # max_nodes is maximum number of nodes
        # num_features is the dimension of the vector space in which we create the embeddings
        
        # set up neural network architecture
        self.input_size = 30 # dimension of normal random variable for sampling
        self.max_nodes = max_nodes # max number of nodes to create. Currently it always creates the maximum number of nodes.
        self.num_features = num_features
        self.output_size = max_nodes * (num_features+2)
        self.hidden_size1 = 10 # number of neurons at hidden layers
        self.hidden_size2 = 10
        self.num_samples = 10
        self.Hbar = Hbars # target proportions 
        self.kern = kern
        self.kern_deriv = kern_deriv
        self.multi = multi
        self.multi_probs = multi_probs
        self.max_nodes_multi = max_nodes_multi
        self.dropout = dropout
        
        # create random starting weights
        self.W1 = th.randn(self.input_size, self.hidden_size1, requires_grad=True)
        self.W2 = th.randn(self.hidden_size1, self.hidden_size2, requires_grad=True)
        self.W3 = th.randn(self.hidden_size2, self.output_size, requires_grad=True)
        
        #####################
        self.f1 = th.randn(self.input_size)
        self.f2 = th.randn(self.input_size)
        #####################
        
    def forward(self, X):        
        # forward pass 
        # leaky relu used instead of regular relu because zero embeddings cause divide by 0 errors
        self.z1 = F.leaky_relu(th.matmul(X, self.W1)) 
        self.z2 = F.leaky_relu(th.matmul(self.z1, self.W2)) 
        self.z3 = th.matmul(self.z2, self.W3)
        return self.z3
    
        
    def train(self, minibatch, gamma, display, bipart, max_order, order_probs, limited_graphlets=0, usable_graphlets3=None, usable_graphlets4=None, usable_graphlets5=None, usable_graphlets6=None,):
        # minibatch is number of samples to take before doing an update
        # gamma is learning rate
        # activate display to put some info onscreen during training
        # order probs is a vector of probabilities for each order of graphlet to be selected
        # bipart is deprecated
        
        if not self.multi:
            sample_nodes1_all = self.better_sampler(self.max_nodes, max_order, minibatch)
            sample_nodes2_all = self.better_sampler(self.max_nodes, max_order, minibatch)
        
        update_W1 = 0
        update_W2 = 0
        update_W3 = 0
        
        for i in range(minibatch):
            # perform a forward pass on two samples of random noise
            sample1_out = self.forward(th.randn(self.input_size))
            sample2_out = self.forward(th.randn(self.input_size))
            sample1_reshape = sample1_out.view(self.max_nodes,-1).detach().numpy()
            sample2_reshape = sample2_out.view(self.max_nodes,-1).detach().numpy()
            
            # error check
            if th.equal(sample1_out, th.zeros(self.max_nodes*self.num_features)) or th.equal(sample2_out, th.zeros(self.max_nodes*self.num_features)):
                print("Error: Samples identical")
                print(sample1_out)
                print(sample2_out)
                print(self.W1,self.W2,self.W3)
            
            # sample a subset of the nodes and an equivalence class of graphlet uniformly
            
            # this line controls how often we sample graphlets of each size. 
            size_sample = np.random.choice([3,4,5,6,7], p=order_probs)

            if size_sample == 3:
                if not self.multi:
                    sample_nodes1 = sample_nodes1_all[i,0:3]
                    sample_nodes2 = sample_nodes2_all[i,0:3]
                # select which graphlet (equivalence class) to work with. 
                if limited_graphlets:
                    idx = floor(len(useable_graphlets3)*np.random.rand())
                    sample_graphlet_ID1 = usable_graphlets3[idx] 
                else:
                    sample_graphlet_ID1 = floor(num_classes_3*np.random.rand()) 
                overall_ID = sample_graphlet_ID1
                for k in range(reverse3.shape[1]):
                    if reverse3[sample_graphlet_ID1,k] == -1: # exploit the data structure to determine how many graphs are in this equivalence class
                        rho1 = k # this is k-1+1
                        if rho1 == 0:
                            print("Error: calculation of rho")
                        break
                sample_L1 = self.int2graph(reverse3[sample_graphlet_ID1,np.random.randint(rho1)],3) # choose a random graph in the equivalence class. In the document, this is the selection of L.
                sample_L2 = self.int2graph(reverse3[sample_graphlet_ID1,np.random.randint(rho1)],3) # choose another.
                Hbar_L1 = self.Hbar[sample_graphlet_ID1] # Get the target proportion of this graphlet
            elif size_sample == 4:
                if not self.multi:
                    sample_nodes1 = sample_nodes1_all[i,0:4]
                    sample_nodes2 = sample_nodes2_all[i,0:4]
                if limited_graphlets:
                    idx = floor(len(useable_graphlets4)*np.random.rand())
                    sample_graphlet_ID1 = usable_graphlets4[idx] 
                else:
                    sample_graphlet_ID1 = floor(num_classes_4*np.random.rand())
                overall_ID = sample_graphlet_ID1+num_classes_3
                for k in range(reverse4.shape[1]):
                    if reverse4[sample_graphlet_ID1,k] == -1:
                        rho1 = k # this is k-1+1
                        if rho1 == 0:
                            print("Error: calculation of rho")
                        break
                sample_L1 = self.int2graph(reverse4[sample_graphlet_ID1,np.random.randint(rho1)],4)
                sample_L2 = self.int2graph(reverse4[sample_graphlet_ID1,np.random.randint(rho1)],4)
                Hbar_L1 = self.Hbar[overall_ID]
            elif size_sample == 5:
                if not self.multi:
                    sample_nodes1 = sample_nodes1_all[i,0:5]
                    sample_nodes2 = sample_nodes2_all[i,0:5]
                if limited_graphlets:
                    idx = floor(len(useable_graphlets5)*np.random.rand())
                    sample_graphlet_ID1 = usable_graphlets5[idx] 
                else:
                    sample_graphlet_ID1 = floor(num_classes_5*np.random.rand()) 
                overall_ID = sample_graphlet_ID1+num_classes_3+num_classes_4
                #sample_graphlet1 = graphs5[:,:,sample_graphlet_ID1]
                for k in range(reverse5.shape[1]):
                    if reverse5[sample_graphlet_ID1,k] == -1:
                        rho1 = k # this is k-1+1
                        if rho1 == 0:
                            print("Error: calculation of rho")
                        break
                sample_L1 = self.int2graph(reverse5[sample_graphlet_ID1,np.random.randint(rho1)],5)
                sample_L2 = self.int2graph(reverse5[sample_graphlet_ID1,np.random.randint(rho1)],5)
                Hbar_L1 = self.Hbar[overall_ID]
            elif size_sample == 6:
                if not self.multi:
                    sample_nodes1 = sample_nodes1_all[i,0:6]
                    sample_nodes2 = sample_nodes2_all[i,0:6]
                if limited_graphlets:
                    idx = floor(len(useable_graphlets6)*np.random.rand())
                    sample_graphlet_ID1 = usable_graphlets6[idx]  
                else:
                    sample_graphlet_ID1 = floor(num_classes_6*np.random.rand()) 
                overall_ID = sample_graphlet_ID1+num_classes_3+num_classes_4+num_classes_5
                #sample_graphlet1 = graphs6[:,:,sample_graphlet_ID1]
                for k in range(reverse6.shape[1]):
                    if reverse6[sample_graphlet_ID1,k] == -1:
                        rho1 = k # this is k-1+1
                        if rho1 == 0:
                            print("Error: calculation of rho")
                        break
                sample_L1 = self.int2graph(reverse6[sample_graphlet_ID1,np.random.randint(rho1)],6)
                sample_L2 = self.int2graph(reverse6[sample_graphlet_ID1,np.random.randint(rho1)],6)
                Hbar_L1 = self.Hbar[overall_ID]
            elif size_sample == 7:
                #TODO: Complete this section (should not be used currently)
                sample_nodes1 = sample_nodes1_all[i,0:7]
                sample_nodes2 = sample_nodes2_all[i,0:7]
                sample_graphlet_ID1 = floor(num_classes_high*np.random.rand()) 
                overall_ID = sample_graphlet_ID1+num_classes_3+num_classes_4+num_classes_5+num_classes_6
                for k in range(reverse_high.shape[1]):
                    if reverse_high[sample_graphlet_ID1,k] == -1:
                        rho1 = k # this is k-1+1
                        if rho1 == 0:
                            print("Error: calculation of rho")
                        break
                sample_L1 = self.int2graph(reverse_high[sample_graphlet_ID1,np.random.randint(rho1)],7)
                sample_L2 = self.int2graph(reverse_high[sample_graphlet_ID1,np.random.randint(rho1)],7)
                Hbar_L1 = self.Hbar[overall_ID]

            # part of the chain rule
            if self.multi:
                prob1 = self.PLA_noA(sample_L1, sample1_reshape, size_sample, self.dropout)
                prob2 = self.PLA_noA(sample_L2, sample2_reshape, size_sample, self.dropout)
                prob_prime1_z = self.PLA_prime_noA(sample_L1, sample1_reshape, size_sample, self.dropout)
                prob_prime2_z = self.PLA_prime_noA(sample_L2, sample2_reshape, size_sample, self.dropout)
            else:
                prob1 = self.PLA(sample_L1, sample1_reshape, sample_nodes1, size_sample, self.dropout) # Use below function to calculate prob of vertex set A having edge set L
                prob2 = self.PLA(sample_L2, sample2_reshape, sample_nodes2, size_sample, self.dropout)
                prob_prime1_z = self.PLA_prime(sample_L1, sample1_reshape, sample_nodes1, size_sample, self.dropout) # 1 x size_sample * num_features
                prob_prime2_z = self.PLA_prime(sample_L2, sample2_reshape, sample_nodes2, size_sample, self.dropout)

            # make it a tensor
            y1 = th.from_numpy(prob_prime1_z)
            y2 = th.from_numpy(prob_prime2_z)
            y1 = y1.type(th.FloatTensor)
            y2 = y2.type(th.FloatTensor)

            # Perform backprop to get gradients for weights
            sample1_out.backward(y1, retain_graph=False)
            W1_update_i1 = self.W1.grad.clone().detach()
            W2_update_i1 = self.W2.grad.clone().detach()
            W3_update_i1 = self.W3.grad.clone().detach()
            
            # zero out the gradients after storing them above
            self.W1.grad.data.zero_()
            self.W2.grad.data.zero_()
            self.W3.grad.data.zero_()
            
            # same thing, other sample
            sample2_out.backward(y2, retain_graph=False)
            W1_update_i2 = self.W1.grad.clone().detach()
            W2_update_i2 = self.W2.grad.clone().detach()
            W3_update_i2 = self.W3.grad.clone().detach()
            self.W1.grad.data.zero_()
            self.W2.grad.data.zero_()
            self.W3.grad.data.zero_()

            # These three lines are critical
            update_W1 += -gamma * (rho1*rho1) * ( W1_update_i1 * (prob2 - (1/rho1)*Hbar_L1) + W1_update_i2 * (prob1 - (1/rho1)*Hbar_L1))
            update_W2 += -gamma * (rho1*rho1) * ( W2_update_i1 * (prob2 - (1/rho1)*Hbar_L1) + W2_update_i2 * (prob1 - (1/rho1)*Hbar_L1))
            update_W3 += -gamma * (rho1*rho1) * ( W3_update_i1 * (prob2 - (1/rho1)*Hbar_L1) + W3_update_i2 * (prob1 - (1/rho1)*Hbar_L1))
            
        # update weights
        with th.no_grad():
            self.W1 += update_W1
            self.W2 += update_W2
            self.W3 += update_W3
                        
        if display:
            #print(self.W1) # activate to display weights
            #print(update_W1) # activate to display update to weights
            sample_size = 100
            sample_graphlets = np.zeros([self.Hbar.size,sample_size])
            difference = np.zeros([self.Hbar.size,sample_size])
            # To get an error we can display, we'll need to generate some graphs using the learned weights and calculate their graphlets
            for i in range(sample_size): # Use 20 graphs to estimate our distribution
                sample_graph = self.generate_graph()
                sample_graphlets[:,i] = self.graph2graphlets(sample_graph, max_order, classes3, classes4, classes5, classes6)
                difference[:,i] = sample_graphlets[:,i] - self.Hbar
            difference_avg = np.mean(difference,axis=1)
            sample_graphlets_avg = np.mean(sample_graphlets,axis=1)
            if max_order == 3:
                total_diff = np.sum(np.absolute(difference_avg[0:4]))
                print(sample_graphlets_avg[0:4])
                print(self.Hbar[0:4])
                print(difference_avg[0:4])
                print(total_diff)
            elif max_order == 4:
                total_diff = np.sum(np.absolute(difference_avg[0:15]))
                print(sample_graphlets_avg[0:15])
                print(self.Hbar[0:15])
                print(difference_avg[0:15])
                print(total_diff)
            elif max_order == 5:
                total_diff = np.sum(np.absolute(difference_avg[0:49]))
                print(sample_graphlets_avg[0:49])
                print(self.Hbar[0:49])
                print(difference_avg[0:49])
                print(total_diff)
            else:
                print("Error: this section of the code is incomplete")         
        else:
            total_diff = 100
        
        return total_diff
    
    def generate_graph(self):
        # generates an actual graph using the results from the neural net
        num_nodes = self.max_nodes
        adj = th.zeros(num_nodes,num_nodes) # adjacency matrix
        
        # perform a forward pass to get embeddings
        with th.no_grad():
            forward_pass_flat = self.forward(th.randn(self.input_size))
            #forward_pass_flat = self.forward(th.randn(self.input_size)+100*th.ones(self.input_size))
            #forward_pass_flat = self.forward(self.f1)
            forward_pass = forward_pass_flat.view(self.max_nodes,self.num_features).detach()
        
        for i in range(num_nodes):
            for j in range(i+1,num_nodes):
                # calculate edge probs
                p = self.kern(forward_pass[i,:],forward_pass[j,:])
                # realize a graph from probs
                rand = uniform(0, 1)
                if rand < p:
                    adj[i,j] = 1
                    adj[j,i] = 1
        return adj
    

    def hv(self,x):
        # hv stands for heaviside even though this is not quite the heaviside function
        # needed for derivative
        if x >= 0:
            return 1
        else:
            return -1
        
    def sigmoid(self, sigm):
        return 1 / (1 + exp(-sigm))
    
    def sigm_deriv(self,sigm):
        #if sigm > 
        return exp(-sigm)/(exp(-sigm)+1)**2
    
    def int2graph(self, I, n):
        # converts an integer I to an n x n graph

        binary = "{0:b}".format(I)[::-1]
        
        m = int(n*(n-1)/2)
        num_zeros_to_add = m - len(binary)
        zeros_to_add = "0"*num_zeros_to_add
        binary = binary + zeros_to_add

        graph = np.zeros([n,n])
        count = 0
        for j in range(1,n):
            for i in range(j):
                graph[i,j] = int(binary[count])
                graph[j,i] = graph[i,j]
                count += 1
        return graph
    
    #@jit
    def graph2int(self,adj):
    # converts a graph whose adjacency matrix is adj into a number between 0 and 2^(n*(n-1)/2) - 1 using binary representation
    # same function as above
    
        n = adj.shape[0]
        bin_vector = np.array([adj[0,1]])

        for i in range(2,n):
            appendage = adj[0:i,i]
            bin_vector = np.append(bin_vector, appendage)

        out = 0
        for i in range(bin_vector.size):
            out += int(2**i * bin_vector[i])

        return out
        
    #@jit
    def graph2graphlets(self, adj, max_order, graphlet_list_3, graphlet_list_4, graphlet_list_5, graphlet_list_6, counts_or_prop=0):
        # converts a graph given in adj to a list of graphlet proportions via sampling
        
        n = adj.shape[0]

        graphlets_2 = np.zeros([2,1])
        graphlets_3 = np.zeros([4,1])
        graphlets_4 = np.zeros([11,1])
        graphlets_5 = np.zeros([34,1])
        graphlets_6 = np.zeros([156,1])


        # Count the 2-graphlets ie. edges
        #num_2 = n*(n-1)/2 
        #graphlets_2[0] = (np.sum(adj)/2) / num_2 
        #graphlets_2[1] = (num_2 - graphlets_2[0]) / num_2


        # Count the 3-graphlets (via sampling)
        num_3 = scipy.special.binom(n,3)
        num_samples_3 = 100
        
        nodes_all = self.better_sampler(n, 3, num_samples_3)

        for i in range(num_samples_3):
            # randomly select 3 nodes from the list, check what graph they form, check its equivalence class, add to count
            nodes = nodes_all[i,:]
            sub_adj = adj[nodes[:, None], nodes]
            sub_num = self.graph2int(sub_adj)
            graphlet_ID = graphlet_list_3[sub_num]
            graphlets_3[graphlet_ID] += 1

        graphlets_3 = graphlets_3 / num_samples_3
        if counts_or_prop:
            graphlets_3 = graphlets_3 * num_3

        if max_order >= 4:
            # Count the 4-graphlets (via sampling)
            num_4 = scipy.special.binom(n,4)
            num_samples_4 = 300
            nodes_all = self.better_sampler(n, 4, num_samples_4)

            for i in range(num_samples_4):
                # randomly select 3 nodes from the list
                nodes = nodes_all[i,:]
                sub_adj = adj[nodes[:, None], nodes]
                sub_num = self.graph2int(sub_adj)
                graphlet_ID = graphlet_list_4[sub_num]
                graphlets_4[graphlet_ID] += 1

            graphlets_4 = graphlets_4 / num_samples_4
            if counts_or_prop:
                graphlets_4 = graphlets_4 * num_4

        if max_order >= 5:
            # Count the 5-graphlets (via sampling)
            num_5 = scipy.special.binom(n,5)
            num_samples_5 = 900
            nodes_all = self.better_sampler(n, 5, num_samples_5)

            for i in range(num_samples_5):
                # randomly select 5 nodes from the list
                nodes = nodes_all[i,:]
                sub_adj = adj[nodes[:, None], nodes]
                sub_num = self.graph2int(sub_adj)
                graphlet_ID = graphlet_list_5[sub_num]
                graphlets_5[graphlet_ID] += 1

            graphlets_5 = graphlets_5 / num_samples_5
            if counts_or_prop:
                graphlets_5 = graphlets_5 * num_5

        if max_order >= 6:
            # Count the 6-graphlets (via sampling)
            num_6 = scipy.special.binom(n,6)
            num_samples_6 = 4500
            nodes_all = self.better_sampler(n, 6, num_samples_6)

            for i in range(num_samples_6):
                # randomly select 6 nodes from the list
                nodes = nodes_all[i,:]
                sub_adj = adj[nodes[:, None], nodes]
                sub_num = self.graph2int(sub_adj)
                graphlet_ID = graphlet_list_6[sub_num]
                graphlets_6[graphlet_ID] += 1

            graphlets_6 = graphlets_6 / num_samples_6
            if counts_or_prop:
                graphlets_6 = graphlets_6 * num_6
    

        # activate below line to use 2-graphlets also
        #graphlets = np.concatenate((graphlets_2,graphlets_3,graphlets_4,graphlets_5,graphlets_6)).ravel() 
        graphlets = np.concatenate((graphlets_3,graphlets_4,graphlets_5,graphlets_6)).ravel()
        return graphlets
    
    def better_sampler(self, maxi, length, num_samples):
        y = np.arange(maxi)
        x = np.zeros([num_samples,length])
        for i in range(num_samples):
            np.random.shuffle(y)
            x[i,:] = y[0:length]
        return x.astype(int)

    def PLA(self, adj, embeddings, node_selections, n, drop=0):
        # function to compute the probability that a particular set of nodes given in node_selections will form a particular subgraph given by adj
        embeddings_subgraph = embeddings[node_selections,:] # assume each row is a vertex and full row is its embedding. This represents A_j in the doc

        result = 1
        for i in range(n):
            for j in range(i+1,n):
                if adj[i,j] == 1: # multiply by edge prob
                    #if np.linalg.norm(embeddings_subgraph[i,:]) == 0:
                    #    print("Error: Norm still too small")
                    result = result*self.kern(embeddings_subgraph[i,2:], embeddings_subgraph[j,2:])
                elif adj[i,j] == 0: # multiply by 1 - edge prob
                    #if np.linalg.norm(embeddings_subgraph[i,:]) == 0:
                    #    print("Error: Norm still too small")
                    result = result*(1-self.kern(embeddings_subgraph[i,2:], embeddings_subgraph[j,2:]))
            if drop:
                result = result*self.sigmoid(embeddings_subgraph[i,1])
        return result
    
    def PLA_noA(self, adj, embeddings, n, drop=0):
        # As above, but used for community model. Directly computes probability without sampling A. 
        num_comms = embeddings.shape[0]
        comms = self.comm_list_maker2(num_comms,n)
        total = 0
                
        for t in range(len(comms)):
            result = 1
            for i in range(n):
                for j in range(i+1,n):
                    if adj[i,j] == 1: # multiply by edge prob
                        result = result*self.kern(embeddings[comms[t][i],2:], embeddings[comms[t][j],2:])
                    elif adj[i,j] == 0: # multiply by 1 - edge prob
                        result = result*(1-self.kern(embeddings[comms[t][i],2:], embeddings[comms[t][j],2:]))
                if drop:
                    result = result*self.sigmoid(embeddings[comms[t][i],1:])
                        
            comm_probs = embeddings[:,0]
            s_max = scipy.special.softmax(comm_probs)
            for i in range(len(comms[t])):
                result = result*s_max[comms[t][i]]
            total += result
        return total
    
    def PLA_alt(self, adj, embeddings, node_selections, n, i_remove, j_remove, drop=0):
        # alternate version of PLA needed to avoid divide by zero issue. Used only in derivative calculation.
        embeddings_subgraph = embeddings[node_selections,:] 
        flag = 1
        
        result = 1
        for i in range(n):
            for j in range(i+1,n):
                if (i == i_remove and j == j_remove) or (j == i_remove and i == j_remove):
                    flag = 0
                    continue
                elif adj[i,j] == 1:
                    result = result*self.kern(embeddings_subgraph[i,2:], embeddings_subgraph[j,2:])
                elif adj[i,j] == 0:
                    result = result*(1-self.kern(embeddings_subgraph[i,2:], embeddings_subgraph[j,2:]))
            if drop:
                result = result*self.sigmoid(embeddings_subgraph[i,1])
        if flag == 1:
            print("Error: ij not removed")
        return result
    
    def PLA_alt2(self, adj, embeddings, node_selections, n, i_remove):
        # alternate version of PLA needed to avoid divide by zero issue. Used only in derivative calculation.
        # this version is to remove the node removal probability
        embeddings_subgraph = embeddings[node_selections,:] 
        flag = 1
        
        result = 1
        for i in range(n):
            for j in range(i+1,n):
                if adj[i,j] == 1:
                    result = result*self.kern(embeddings_subgraph[i,2:], embeddings_subgraph[j,2:])
                elif adj[i,j] == 0:
                    result = result*(1-self.kern(embeddings_subgraph[i,2:], embeddings_subgraph[j,2:]))
                else:
                    print("Error: Adjacency matrix incorrect")
            if i == i_remove:
                flag = 0
                continue
            else:
                result = result*self.sigmoid(embeddings_subgraph[i,1])
            
        if flag == 1:
            print("Error: i not removed")
        return result
    
    def PLA_prime(self, adj, embeddings, node_selections, n, drop=0):
    # output should be 1 x s*k
        
    # derivative of PLA wrt. weights
    
        embeddings_subgraph = embeddings[node_selections,:] # assume each row is a vertex and full row is its embedding
        embeddings_length = embeddings.shape[1]

        del_P_del_z = np.zeros([embeddings.shape[0],embeddings.shape[1]])
        for i in range(n): 
            # for each node, compute derivative of P wrt that node's embedding, multiply by derivative of embedding wrt weights, add up
            total_i = np.zeros([embeddings_length])
            for j in range(n):
                if i == j:
                    continue

                #########################
                embed_i = embeddings_subgraph[i,2:]
                embed_j = embeddings_subgraph[j,2:]
                
                #col vector
                #for k in range(embeddings_length):
                    #phi_ij_prime[k] = (self.hv(phi_ij)*embeddings_subgraph[j,k]*norm_i*norm_j - norm_j*embeddings_subgraph[i,k]*abs(phi_ij)/norm_i) / (norm_i**2 * norm_j**2) # calculated derivative
                phi_ij_prime = self.kern_deriv(embed_i, embed_j)
                #########################
                
                if adj[i,j] == 1:
                    # chain rule
                    total_i += phi_ij_prime * self.PLA_alt(adj,embeddings,node_selections,n,i,j,self.dropout)                   
                elif adj[i,j] == 0:
                    total_i += -phi_ij_prime * self.PLA_alt(adj,embeddings,node_selections,n,i,j,self.dropout)
            
            # derivatives wrt dropout probs
            if drop:
                # can apply a scale factor here or elsewhere if needed
                removal_prob_deriv = self.PLA_alt2(adj,embeddings,node_selections,n,i) * self.sigm_deriv(embeddings_subgraph[i,1])
                total_i = total_i.ravel()
                total_i = np.insert(total_i, 0, [0,removal_prob_deriv])
            else:
                total_i = total_i.ravel()
                total_i = np.insert(total_i, 0, [0,0])
                    
            del_P_del_z[node_selections[i],:] = total_i # put the derivatives in the correct place for each node
             
        del_P_del_z = del_P_del_z.ravel()
        
        return del_P_del_z
    
    def PLA_prime_noA(self, adj, embeddings, n, drop=0):
    # output should be 1 x s*k
    # derivative of PLA wrt. weights
    # As above, this version is for the community model where embeddings are repeated.
    
        num_comms = embeddings.shape[0]
        comms = self.comm_list_maker2(num_comms,n)
        comm_probs = embeddings[:,0]
        s_max = scipy.special.softmax(comm_probs)
        
        embeddings_length = embeddings.shape[1]
        for t in range(len(comms)):
            embeddings_subgraph = embeddings[comms[t],:] # assume each row is a vertex and full row is its embedding

            del_P_del_z = np.zeros([embeddings.shape[0],embeddings.shape[1]])
            for i in range(n): 
                # for each node, compute derivative of P wrt that node's embedding, multiply by derivative of embedding wrt weights, add up
                total_i = np.zeros([embeddings_length-2])
                for j in range(n):
                    if i == j:
                        continue

                    #########################
                    embed_i = embeddings_subgraph[i,2:]
                    embed_j = embeddings_subgraph[j,2:]

                    #col vector
                    #for k in range(embeddings_length):
                        #phi_ij_prime[k] = (self.hv(phi_ij)*embeddings_subgraph[j,k]*norm_i*norm_j - norm_j*embeddings_subgraph[i,k]*abs(phi_ij)/norm_i) / (norm_i**2 * norm_j**2) # calculated derivative
                    phi_ij_prime = self.kern_deriv(embed_i, embed_j)
                    #########################

                    if adj[i,j] == 1:
                        # chain rule
                        total_i += phi_ij_prime * self.PLA_alt(adj,embeddings,comms[t],n,i,j,self.dropout)                   
                    elif adj[i,j] == 0:
                        total_i += -phi_ij_prime * self.PLA_alt(adj,embeddings,comms[t],n,i,j,self.dropout)
                    
                # derivatives wrt dropout probs
                if drop:
                    # can apply a scale factor here or elsewhere if needed
                    removal_prob_deriv = self.PLA_alt2(adj,embeddings,node_selections,n,i) * self.sigm_deriv(embeddings_subgraph[i,1])
                    total_i = total_i.ravel()
                    total_i = np.insert(total_i, 0, [0,removal_prob_deriv])
                else:
                    total_i = total_i.ravel()
                    total_i = np.insert(total_i, 0, [0,0])

                #del_P_del_z[node_selections[i],:] += total_i # put the derivatives in the correct place for each node
                del_P_del_z[comms[t][i],:] += total_i # put the derivatives in the correct place for each node
            
            # This part computes the derivatives wrt each of the community probs
            num_repeats = np.zeros(num_comms)
            comm_prob_deriv_factor = 1
            for i in range(num_comms):
                num_repeats[i] = comms[t].count(i)
                comm_prob_deriv_factor = s_max[i] ** num_repeats[i]
            for i in range(num_comms):
                comm_prob_deriv = num_repeats[i] - n*s_max[i]
                comm_prob_deriv = comm_prob_deriv * comm_prob_deriv_factor
                comm_prob_deriv = comm_prob_deriv * self.PLA(adj, embeddings, comms[t], n, self.dropout)
                del_P_del_z[i,0] += comm_prob_deriv
                # might multiply derivative by a small number (.01 used previously) to make the steps smaller here
                
        #print(del_P_del_z[:,0])
        del_P_del_z = del_P_del_z.ravel()
        
        return del_P_del_z
    
    def comm_list_maker2(self,r,s):
        created_list = []
        for i in range(r**s):
            created_list.append(self.number_to_base(i,r,s))
        return created_list
        
    def number_to_base(self,n,b,r):
        # n = number, b = base, r = length of digits including added zeros
        # currently returns numbers with ones digits first and zeros at the end
        if n == 0:
            return [0]*r
        digits = []
        while n:
            digits.append(int(n % b))
            n //= b
        digits.extend([0]*(r-len(digits)))
        return digits

NameError: name 'nn' is not defined

In [ ]:
# standard dot product kernel 

@jit
def dotprod_kern(em1,em2):
    return abs(np.dot(em1,em2))/(np.linalg.norm(em1)*np.linalg.norm(em2))

def hv(x):
    if x >= 0:
        return 1
    else: 
        return -1
@jit
def dotprod_kern_prime(em1,em2):
    phi_ij = np.dot(em1,em2)
    norm_i = np.linalg.norm(em1)
    norm_j = np.linalg.norm(em2)
    phi_ij_prime = (hv(phi_ij)*em2*norm_i*norm_j - norm_j*em1*abs(phi_ij)/norm_i) / (norm_i**2 * norm_j**2) # calculated derivative
    return phi_ij_prime

In [ ]:
# radial basis function kernel

@jit
def rbf_kern(em1, em2):
    sig = 100
    return exp(-(np.linalg.norm(em1 - em2)**2)/(2*sig**2))


def rbf_kern_prime(em1, em2):
    sig = 100 # may want to adjust this 
    rbf_prime = np.zeros([len(em1),1])
    kern_val = exp(-(np.linalg.norm(em1 - em2)**2)/(2*sig**2))
    rbf_prime = (-1/sig**2)*kern_val*(em1 - em2)
    return rbf_prime

In [ ]:
# rbf kernel with scale factor

#@jit
def rbf2_kern(em1, em2):
    # version of RBF kern with scale factor (1+t^2)^(-1/2) in front
    sig = 10
    scale = 2
    return (1 + np.linalg.norm(em1)**2/scale)**(-1/2) * (1 + np.linalg.norm(em2)**2/scale)**(-1/2) * exp(-(np.linalg.norm(em1 - em2)**2)/(2*sig**2))

#@jit
def rbf2_kern_prime(em1, em2):
    sig = 10
    scale = 2
    rbf_prime = np.zeros([len(em1),1])
    kern_val_Fx = (1 + np.linalg.norm(em1)**2/2)**(-1/2)
    kern_val_Fy = (1 + np.linalg.norm(em2)**2/2)**(-1/2)
    kern_val_G = exp(-(np.linalg.norm(em1 - em2)**2)/(2*sig**2))
    rbf_prime = kern_val_Fy * ( (-em1/scale*kern_val_Fx**3)*kern_val_G + (-1/sig**2)*kern_val_G*(em1 - em2)*kern_val_Fx )
    return rbf_prime

In [ ]:
def better_sampler(maxi, length, num_samples, no_replace=0):
    y = np.arange(maxi)
    x = np.zeros([num_samples,length])
    if no_replace:
        x = np.floor(maxi*np.random.rand(num_samples,length))
    else:
        for i in range(num_samples):
            np.random.shuffle(y)
            x[i,:] = y[0:length]
    return x.astype(int)

In [ ]:
# a function which converts a graph to its graphlets via sampling

#@jit
def graph2graphlets(adj, max_order, graphlet_list_3, graphlet_list_4, graphlet_list_5, graphlet_list_6, counts_or_prop=0):
    # converts a graph given in adj to a list of graphlet proportions via sampling

    n = adj.shape[0]

    graphlets_2 = np.zeros([2,1])
    graphlets_3 = np.zeros([4,1])
    graphlets_4 = np.zeros([11,1])
    graphlets_5 = np.zeros([34,1])
    graphlets_6 = np.zeros([156,1])


    # Count the 2-graphlets ie. edges
    #num_2 = n*(n-1)/2 
    #graphlets_2[0] = (np.sum(adj)/2) / num_2 
    #graphlets_2[1] = (num_2 - graphlets_2[0]) / num_2


    # Count the 3-graphlets (via sampling)
    num_3 = scipy.special.binom(n,3)
    num_samples_3 = 100

    nodes_all = better_sampler(n, 3, num_samples_3)

    for i in range(num_samples_3):
        # randomly select 3 nodes from the list, check what graph they form, check its equivalence class, add to count
        nodes = nodes_all[i,:]
        sub_adj = adj[nodes[:, None], nodes]
        sub_num = graph2int(sub_adj)
        graphlet_ID = graphlet_list_3[sub_num]
        graphlets_3[graphlet_ID] += 1

    graphlets_3 = graphlets_3 / num_samples_3
    if counts_or_prop:
        graphlets_3 = graphlets_3 * num_3

    if max_order >= 4:
        # Count the 4-graphlets (via sampling)
        num_4 = scipy.special.binom(n,4)
        num_samples_4 = 300
        nodes_all = better_sampler(n, 4, num_samples_4)

        for i in range(num_samples_4):
            # randomly select 3 nodes from the list
            nodes = nodes_all[i,:]
            sub_adj = adj[nodes[:, None], nodes]
            sub_num = graph2int(sub_adj)
            graphlet_ID = graphlet_list_4[sub_num]
            graphlets_4[graphlet_ID] += 1

        graphlets_4 = graphlets_4 / num_samples_4
        if counts_or_prop:
            graphlets_4 = graphlets_4 * num_4

    if max_order >= 5:
        # Count the 5-graphlets (via sampling)
        num_5 = scipy.special.binom(n,5)
        num_samples_5 = 900
        nodes_all = better_sampler(n, 5, num_samples_5)

        for i in range(num_samples_5):
            # randomly select 5 nodes from the list
            nodes = nodes_all[i,:]
            sub_adj = adj[nodes[:, None], nodes]
            sub_num = graph2int(sub_adj)
            graphlet_ID = graphlet_list_5[sub_num]
            graphlets_5[graphlet_ID] += 1

        graphlets_5 = graphlets_5 / num_samples_5
        if counts_or_prop:
            graphlets_5 = graphlets_5 * num_5

    if max_order >= 6:
        # Count the 6-graphlets (via sampling)
        num_6 = scipy.special.binom(n,6)
        num_samples_6 = 4500
        nodes_all = better_sampler(n, 6, num_samples_6)

        for i in range(num_samples_6):
            # randomly select 6 nodes from the list
            nodes = nodes_all[i,:]
            sub_adj = adj[nodes[:, None], nodes]
            sub_num = graph2int(sub_adj)
            graphlet_ID = graphlet_list_6[sub_num]
            graphlets_6[graphlet_ID] += 1

        graphlets_6 = graphlets_6 / num_samples_6
        if counts_or_prop:
            graphlets_6 = graphlets_6 * num_6


    # activate below line to use 2-graphlets also
    #graphlets = np.concatenate((graphlets_2,graphlets_3,graphlets_4,graphlets_5,graphlets_6)).ravel() 
    graphlets = np.concatenate((graphlets_3,graphlets_4,graphlets_5,graphlets_6)).ravel()
    return graphlets

In [ ]:
def graph2int(adj):
    # converts a graph whose adjacency matrix is adj into a number between 0 and 2^(n*(n-1)/2) - 1 using binary representation
    # same function as above
    
    n = adj.shape[0]
    bin_vector = np.array([adj[0,1]])
    
    for i in range(2,n):
        appendage = adj[0:i,i]
        bin_vector = np.append(bin_vector, appendage)
    
    out = 0
    for i in range(bin_vector.size):
        out += int(2**i * bin_vector[i])
        
    return out

In [ ]:
def int2graphs(I, n):
        # converts an integer I to an n x n graph

        binary = "{0:b}".format(I)[::-1]

        m = int(n*(n-1)/2)
        num_zeros_to_add = m - len(binary)
        zeros_to_add = "0"*num_zeros_to_add
        binary = binary + zeros_to_add

        graph = np.zeros([n,n])
        count = 0
        for j in range(1,n):
            for i in range(j):
                graph[i,j] = int(binary[count])
                graph[j,i] = graph[i,j]
                count += 1
        return graph

# Datasets

In [ ]:
# AIDS dataset

# load the data

# There is a lot of preprocessing that happens here to get the data into a usable form

A = []
with open("aids_A.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        A.append(row)
        
A[0][0] = 1
A = np.array(A)
A = A.astype(np.int)


graph_indicator = []
with open("aids_indicator.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        graph_indicator.append(row)
        
graph_indicator[0][0] = 1
graph_indicator = np.array(graph_indicator)
graph_indicator = graph_indicator.astype(np.int)


graph_labels = []
with open("aids_labels.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        graph_labels.append(row)
        
graph_labels[0][0] = 0
graph_labels = np.array(graph_labels)
graph_labels = graph_labels.astype(np.int)

#print(graph_labels)
#with open('aids_indicator.csv', newline='') as csvfile:
#    graph_indicator = list(csv.reader(csvfile))


graphs = []

start_points = [0]
end_points = []
for i in range(1,2001):
    for j in range(start_points[i-1],31385):
        if j == 31384:
            end_points.append(j+2)
            num_nodes = end_points[i-1] - start_points[i-1]
            graphs.append(np.zeros([num_nodes,num_nodes]))
        if graph_indicator[j,0] > i:
            end_points.append(j+1)
            #print(end_points)
            if i != 2000:
                start_points.append(j+1)
            num_nodes = end_points[i-1] - start_points[i-1]
            graphs.append(np.zeros([num_nodes,num_nodes]))
            break
            
for i in range(64780):
    point1 = A[i,0]
    point2 = A[i,1]
    for j in range(2000):
        if end_points[j] >= point1 and end_points[j] >= point2:
            graph_num = j
            break
    #print(start_points)
    #print(point1,point2,graph_num,start_points[graph_num],end_points[graph_num])
    
    graphs[graph_num][point1-start_points[graph_num], point2-start_points[graph_num]] = 1
    graphs[graph_num][point2-start_points[graph_num], point1-start_points[graph_num]] = 1
    
#shuffled = list(zip(graphs,graph_labels))
#shuffle(shuffled)
#graphs, graph_labels = zip(*shuffled)


In [ ]:
# create Hbar for AIDS dataset.

Hbar_avg_AIDS = 0
j = 0
for i in range(3000):
    graph_number = np.random.randint(2000)
    if graph_labels[graph_number,0] == 1:
        continue
    else:
        Hbar = graph2graphlets(graphs[graph_number], 4, classes3, classes4, classes5, classes6)
        j += 1
        Hbar_avg_AIDS = Hbar_avg_AIDS*(j-1)/(j) + 1/(j)*Hbar

print(Hbar_avg_AIDS)

In [ ]:
# COX2_MD dataset

# load the data

# There is a lot of preprocessing that happens here to get the data into a usable form

A = []
with open("cox2_md_A.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        A.append(row)
        
A[0][0] = 1
A = np.array(A)
A = A.astype(np.int)


graph_indicator = []
with open("cox2_md_indicator.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        graph_indicator.append(row)
        
graph_indicator[0][0] = 1
graph_indicator = np.array(graph_indicator)
graph_indicator = graph_indicator.astype(np.int)


graph_labels = []
with open("cox2_md_labels.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        graph_labels.append(row)
        
graph_labels[0][0] = 0
graph_labels = np.array(graph_labels)
graph_labels = graph_labels.astype(np.int)

#print(graph_labels)
#with open('aids_indicator.csv', newline='') as csvfile:
#    graph_indicator = list(csv.reader(csvfile))


graphs = []

start_points = [0]
end_points = []
for i in range(1,304): # one more than the number of graphs in the dataset
    for j in range(start_points[i-1],graph_indicator.shape[0]): # number of nodes
        if j == graph_indicator.shape[0]-1: # one fewer than the number of nodes ie. the number of rows in graph_indicator
            end_points.append(j+2)
            num_nodes = end_points[i-1] - start_points[i-1]
            graphs.append(np.zeros([num_nodes,num_nodes]))
        if graph_indicator[j,0] > i:
            end_points.append(j+1)
            #print(end_points)
            if i != 303:
                start_points.append(j+1)
            num_nodes = end_points[i-1] - start_points[i-1]
            graphs.append(np.zeros([num_nodes,num_nodes]))
            break
            
for i in range(A.shape[0]): # the number of edges, ie. the number of rows in A
    point1 = A[i,0]
    point2 = A[i,1]
    for j in range(303):
        if end_points[j] >= point1 and end_points[j] >= point2:
            graph_num = j
            break
    #print(start_points)
    #print(point1,point2,graph_num,start_points[graph_num],end_points[graph_num])
    
    graphs[graph_num][point1-start_points[graph_num], point2-start_points[graph_num]] = 1
    graphs[graph_num][point2-start_points[graph_num], point1-start_points[graph_num]] = 1


In [ ]:
# create Hbar for COX2_MD dataset.

Hbar_avg_COX = 0
j = 0
for i in range(3000): # does not depend on data
    graph_number = np.random.randint(303) # number of graphs
    if graph_labels[graph_number,0] == 1:
        continue
    else:
        Hbar = graph2graphlets(graphs[graph_number],4, classes3, classes4, classes5, classes6)
        j += 1
        Hbar_avg_COX = Hbar_avg_COX*(j-1)/(j) + 1/(j)*Hbar

print(Hbar_avg_COX)

In [ ]:
# OHSU dataset

# load the data

# There is a lot of preprocessing that happens here to get the data into a usable form

A = []
with open("ohsu_A.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        A.append(row)
        
A[0][0] = 1
A = np.array(A)
A = A.astype(np.int)


graph_indicator = []
with open("ohsu_indicator.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        graph_indicator.append(row)
        
graph_indicator[0][0] = 1
graph_indicator = np.array(graph_indicator)
graph_indicator = graph_indicator.astype(np.int)


graph_labels = []
with open("ohsu_labels.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        graph_labels.append(row)
        
graph_labels[0][0] = 0
graph_labels = np.array(graph_labels)
graph_labels = graph_labels.astype(np.int)

#print(graph_labels)
#with open('aids_indicator.csv', newline='') as csvfile:
#    graph_indicator = list(csv.reader(csvfile))


graphs = []

start_points = [0]
end_points = []
for i in range(1,80): # one more than the number of graphs in the dataset
    for j in range(start_points[i-1],graph_indicator.shape[0]): # number of edges
        if j == graph_indicator.shape[0]-1: # one fewer than the number of edges ie. the number of rows in graph_indicator
            end_points.append(j+2)
            num_nodes = end_points[i-1] - start_points[i-1]
            graphs.append(np.zeros([num_nodes,num_nodes]))
        if graph_indicator[j,0] > i:
            end_points.append(j+1)
            #print(end_points)
            if i != 79: # number of graphs
                start_points.append(j+1)
            num_nodes = end_points[i-1] - start_points[i-1]
            graphs.append(np.zeros([num_nodes,num_nodes]))
            break
            
for i in range(A.shape[0]): # the number of edges, ie. the number of rows in A
    point1 = A[i,0]
    point2 = A[i,1]
    for j in range(79): # number of graphs
        if end_points[j] >= point1 and end_points[j] >= point2:
            graph_num = j
            break
    #print(start_points)
    #print(point1,point2,graph_num,start_points[graph_num],end_points[graph_num])
    
    graphs[graph_num][point1-start_points[graph_num], point2-start_points[graph_num]] = 1
    graphs[graph_num][point2-start_points[graph_num], point1-start_points[graph_num]] = 1


In [ ]:
# create Hbar for OHSU dataset.

Hbar_avg_OHSU = 0
j = 0
for i in range(1000): # does not depend on data
    graph_number = np.random.randint(79) # number of graphs
    if graph_labels[graph_number,0] == 1:
        continue
    else:
        Hbar = graph2graphlets(graphs[graph_number], classes3, classes4, classes5, classes6)
        j += 1
        Hbar_avg_OHSU = Hbar_avg_OHSU*(j-1)/(j) + 1/(j)*Hbar

print(Hbar_avg_OHSU)

In [ ]:
# Proteins dataset

# load the data

# There is a lot of preprocessing that happens here to get the data into a usable form

A = []
with open("proteins_A.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        A.append(row)
        
A[0][0] = 12
A = np.array(A)
A = A.astype(np.int)


graph_indicator = []
with open("proteins_indicator.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        graph_indicator.append(row)
        
graph_indicator[0][0] = 1
graph_indicator = np.array(graph_indicator)
graph_indicator = graph_indicator.astype(np.int)


graph_labels = []
with open("proteins_labels.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        graph_labels.append(row)
        
graph_labels[0][0] = 0
graph_labels = np.array(graph_labels)
graph_labels = graph_labels.astype(np.int)



number_graphs = 1113
number_nodes = graph_indicator.shape[0] #43471
number_edges = 162088

graphs = []

start_points = [0]
end_points = []
for i in range(1,number_graphs+1):
    for j in range(start_points[i-1],number_nodes):
        if j == number_nodes-1:
            end_points.append(j+2)
            num_nodes = end_points[i-1] - start_points[i-1]
            graphs.append(np.zeros([num_nodes,num_nodes]))
        if graph_indicator[j,0] > i:
            end_points.append(j+1)
            #print(end_points)
            if i != number_graphs:
                start_points.append(j+1)
            num_nodes = end_points[i-1] - start_points[i-1]
            graphs.append(np.zeros([num_nodes,num_nodes]))
            break
                        
for i in range(number_edges):
    point1 = A[i,0]
    point2 = A[i,1]
    for j in range(number_graphs):
        if end_points[j] >= point1 and end_points[j] >= point2:
            graph_num = j
            break
    #print(start_points)
    #print(point1,point2,graph_num,start_points[graph_num],end_points[graph_num])
    
    graphs[graph_num][point1-start_points[graph_num], point2-start_points[graph_num]] = 1
    graphs[graph_num][point2-start_points[graph_num], point1-start_points[graph_num]] = 1
    
#shuffled = list(zip(graphs,graph_labels))
#shuffle(shuffled)
#graphs, graph_labels = zip(*shuffled)


In [ ]:
Hbar_avg_proteins = 0
j = 0
for i in range(3000):
    graph_number = np.random.randint(number_graphs)
    if graph_labels[graph_number,0] == 1 or graphs[graph_number].shape[0] <= 4:
        continue
    else:
        Hbar = graph2graphlets(graphs[graph_number], 4, classes3, classes4, classes5, classes6)
        j += 1
        Hbar_avg_proteins = Hbar_avg_proteins*(j-1)/(j) + 1/(j)*Hbar

print(Hbar_avg_proteins)

In [ ]:
# Brain dataset
# 114 graphs, 70 nodes each

graphs = []

for i in range(114):
    A = []
    filename = "carey_data/carey_data_" + str(i) + ".csv"
    with open(filename) as csvfile:
        reader = csv.reader(csvfile) # change contents to floats
        for row in reader: # each row is a list
            A.append(row)
    A = np.array(A)
    A = A[1:,1:]
    A = A.astype(np.int)
    graphs.append(A)


In [ ]:
# create Hbar for Brain dataset.

Hbar_avg_carey = 0
j = 0
for i in range(1000): # does not depend on data
    graph_number = 2*np.random.randint(57) # number of graphs
    Hbar = graph2graphlets(graphs[graph_number],4, classes3, classes4, classes5, classes6)
    j += 1
    Hbar_avg_carey = Hbar_avg_carey*(j-1)/(j) + 1/(j)*Hbar

print(Hbar_avg_carey)

In [ ]:
# the empty graph to try to learn

adj3 = np.zeros([10,10])
Hbar_avg3 = graph2graphlets(adj3, classes3, classes4, classes5, classes6)

In [ ]:
# A simple SBM to try to learn

group_probs = np.array([.25,.25,.25,.25]) # block membership probs
edge_probs = np.array([[.75,.1,.1,.1],[.1,.75,.1,.1],[.1,.1,.75,.1],[.1,.1,.1,.75]]) # edge probs by block

num_nodes_sbm = 16

H_sbm = 0
for i in range(200):
    assignments = np.random.randint(low=0,high=4,size=num_nodes_sbm)
    adj_sbm = np.zeros([num_nodes_sbm,num_nodes_sbm])
    for j in range(num_nodes_sbm):
        for k in range(j+1,num_nodes_sbm):
            edge_prob = edge_probs[assignments[j]][assignments[k]]
            rand = uniform(0, 1)
            if rand < edge_prob:
                adj_sbm[j,k] = 1
                adj_sbm[k,j] = 1
    H_sbm_iter = graph2graphlets(adj_sbm,4,classes3,classes4,classes5,classes6)
    H_sbm += (1/200)*H_sbm_iter
            

In [ ]:
# The "Community" graph from the GraphRNN paper

group_probs = np.array([.5,.5]) # block membership probs
edge_probs = np.array([[.3,.05],[.05,.3]]) # edge probs by block
#edge_probs = np.array([[1.0,.0],[.0,1.0]])

num_nodes_sbm = 80
num_graphs_sbm = 500

H_sbm = 0
for i in range(num_graphs_sbm):
    assignments = np.random.randint(low=0,high=2,size=num_nodes_sbm)
    adj_sbm = np.zeros([num_nodes_sbm,num_nodes_sbm])
    for j in range(num_nodes_sbm):
        for k in range(j+1,num_nodes_sbm):
            edge_prob = edge_probs[assignments[j]][assignments[k]]
            rand = uniform(0, 1)
            if rand < edge_prob:
                adj_sbm[j,k] = 1
                adj_sbm[k,j] = 1
    H_sbm_iter = graph2graphlets(adj_sbm,4,classes3,classes4,classes5,classes6)
    H_sbm += (1/num_graphs_sbm)*H_sbm_iter
    
lims3 = np.argpartition(H_sbm[0:4], -2)[-2:]
lims4 = np.argpartition(H_sbm[4:15], -6)[-6:]
lims5 = np.argpartition(H_sbm[15:], -10)[-10:]

# Run Code

In [ ]:
# Create and train an instance of the neural net

device = th.device("cuda:0" if th.cuda.is_available() else "cpu")
print(device)

#max_nodes = 2 # 16 is average nodes for AIDS dataset
max_nodes = 10
num_features = 8 # dimension of unit ball. depending on kernel, needs to be bigger than the number of nodes or some dists will be impossible
max_nodes_multiple = 10000 # the actual number of nodes needed when using multi functionality
minibatch = 100
order = 4
limited = 0 


net_generate2 = graph_learner(H_sbm, max_nodes, num_features, rbf_kern, rbf_kern_prime)
# optional arguments copied for convenience
# multi=0, multi_probs=[], max_nodes_multi=0, dropout=0)

net_generate2.to(device)

max_epoch = 20000 #200
gamma_start = 1
gammas = gamma_start

stop_threshold = .2 # total deviation from target distribution

result = np.array([100])
flag1 = 0
flag2 = 0

for epoch in range(max_epoch):
    print(epoch)

    if result[-1] < .5 and flag1 == 0:
        gammas = gammas/10
        flag1 = 1
    if result[-1] < .2 and flag2 == 0 and flag1 == 1:
        gammas = gammas/2
        flag2 = 1
        
    if epoch % 10 == 0:
        print(epoch)
        net_out = net_generate2.train(minibatch, gammas, 1, 1, order, [.5,.5,0,0,0], limited)
        result = np.append(result,net_out)
    else:
        net_out = net_generate2.train(minibatch, gammas, 0, 1, order, [.5,.5,0,0,0], limited)
    
    if net_out < stop_threshold:
        print(epoch)
        print(net_out)
        result = np.append(result,net_out)
        break

# negative result = too few graphlets of that type
# positive = too many of that type